# Exercício 5

#### Victor Marcius Magalhães Pinto

## Descrição do Problema

Considerando dados sintéticos, gerados para problemas não-linearmente separáveis, e.g. espirais, utilize o classificador Bayesiano para resolver os problemas de classificação correspondentes, utilizando mistura de Gaussianas com clustering e o KDE estimado pelo método do Silverman. Aplicar os dois métodos para o problema do Golub e comparar os resultados com aqueles obtidos considerando uma única Gaussiana por classe. 

## Testando métodos para dados sintéticos

### Gerando dados

Inicialmente vamos preparar o ambiente do notebook utilizado para os experimentos deste exercício. Os pacotes necessários serão importados a seguir.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, f1_score, recall_score, precision_score, roc_curve
from sklearn.datasets import make_moons
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import random
import pandas as pd
import os
from os.path import join
import warnings
from tqdm import tqdm
from IPython.core.debugger import set_trace
warnings.filterwarnings('ignore')

Primeiro vamos implementar a função geradora de dados em espiral, a ser utilizada como testbench para o classificador por mistura de gaussianas. Agradecimentos aqui ao Henrico Barbosa pela sugestão de função geradora.

In [ ]:
def twospirals(n_points, n_turns, ts=np.pi, tinc=1, noise=.3):
    """
     Returns the two spirals dataset.
     modificado de: 
         https://glowingpython.blogspot.com/2017/04/solving-two-spirals-problem-with-keras.html
    Primeiro gera uma espiral e obtem a segunda espelhando a primeira
    """
    # equação da espiral (coord polares): r = tinc*theta
    # n_points: número de pontos de cada espiral
    # n_turns: número de voltas das espirais
    # ts: ângulo inicial da espiral em radianos
    # tinc: taxa de crescimento do raio em função do ângulo
    # noise: desvio-padrão do ruído
    
    # Sorteando aleatoriamente pontos da espiral
    n = np.sqrt(np.random.rand(n_points,1))  #intervalo [0,1] equivale a [0,theta_max]
                                             #tomar a raiz quadrada ajuda a 
                                             #distribuir melhor os pontos
    ns = (ts)/(2*np.pi*n_turns) #ponto do intervalo equivalente a ts radianos
    n = ns + n_turns*n # intervalo [ns,ns+n_turns] equivalente a [ts, theta_max]
    n = n*(2*np.pi) #intervalo [ts, theta_max]
    
    # Espiral 1
    d1x = np.cos(n)*tinc*n + np.random.randn(n_points,1) * noise
    d1y = np.sin(n)*tinc*n + np.random.randn(n_points,1) * noise
    
    # Espiral 2
    d2x = -np.cos(n)*tinc*n + np.random.randn(n_points,1) * noise
    d2y = -np.sin(n)*tinc*n + np.random.randn(n_points,1) * noise
    
    spirals_points = np.vstack((np.hstack((d1x,d1y)),np.hstack((d2x,d2y))))
    points_labels = np.hstack((np.ones(n_points),np.zeros(n_points)))
    return (spirals_points, points_labels)

Geramos a seguir os dados utilizados para treinamento do classificador, que consiste na obtenção dos clusters e nas estatísticas associadas a cada um deles. Tembém geramos os dados de teste.

In [ ]:
x, y = twospirals(800,2,ts=np.pi,tinc=1,noise=0.5)
x_test, y_test = twospirals(1000,2,ts=np.pi,tinc=1,noise=0.5)

a distribuição dos dados de treinamento:

In [ ]:
plt.figure()
plt.title('Data Set Treinamento')
plt.plot(x[y==0,0], x[y==0,1], '.', label='Classe 1')
plt.plot(x[y==1,0], x[y==1,1], '.', label='Classe 2')
plt.legend()
plt.show()

e a distribuição dos dados de teste:

In [ ]:
plt.figure()
plt.title('Data Set Teste')
plt.plot(x_test[y_test==0,0], x_test[y_test==0,1], '.', label='Classe 1')
plt.plot(x_test[y_test==1,0], x_test[y_test==1,1], '.', label='Classe 2')
plt.legend()
plt.show()

Implementamos agora o classificador que faz uso de mistura de gaussianas para a separação de classes não-linearmente separáveis, como as dos conjuntos obtidos acima. 

In [ ]:
class GaussianMixture:
    
    def __init__(self, clusters=(3, 3)):
        """
        Gaussian mixture Model.
        Args:
            custers: number of clusters to use for each class. Considering binary problems, 
                it is a list (or tuple) with two int values.
        """
        self.clusters = clusters
    
    def calc_statistics(self, x):
        """
        Calculate mean and covariance matrix for a distribution.
        Args:
            x: data distribution.
        Returns:
            Mean and covariance matrix for current distribution.
        """
        mean_1 = np.mean(x, axis=0)
        sigma_1 = np.cov(x.T)
        
        return mean_1, sigma_1
    
    def get_statistics(self, x, k):
        """
        Get statistics for gaussian mixture for a distribution.
        Args:
            x: Data distribution.
            k: number of clusters to split data.
        Returns:
            A list of dictionaries, for each of the k clusters, with its mean, covariance matrix and
            cluster probability.
        """
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(x)
        N = x.size
        cluster_stat = []
        for cluster in range(k):
            c_samples = kmeans.labels_ == cluster
            N_k = x[c_samples].shape[0]
            pi_k = N_k / N
            mean, sigma = self.calc_statistics(x[c_samples])
            cluster_stat.append(
                {
                    'mean': mean, 
                    'sigma': sigma, 
                    'pi_k': pi_k
                }
            )
        return cluster_stat
    
    def fit(self, x_train, y_train):
        """
        Fit the gaussian mixture model.
        Args:
            x_train: Training samples.
            y_train: Training samples labels.
        """
        labels = list(Counter(y_train).keys())
        self.labels = {'class_1': labels[0], 'class_2': labels[1]}
        
        p_class_1 = x_train[y_train == labels[0], :].shape[0]
        p_class_2 = x_train[y_train == labels[1], :].shape[0]

        self.k = p_class_1 / p_class_2
        
        self.statistics_c1 = self.get_statistics(x_train[y_train == labels[0], :], self.clusters[0])
        self.statistics_c2 = self.get_statistics(x_train[y_train == labels[1], :], self.clusters[1])

    def likelihood(self, x, mean, sigma):
        """
        Calculate likelihood of a sample with a gaussian distribution.
        Args:
            x: Sample.
            mean: Mean of the gaussian distribution.
            sigma: Covariance matrixof the gaussian distribution.
        Returns:
            The likehood value.
        """
        n = mean.shape[0]
        arg = (x - mean)
        term = np.matmul(np.matmul(arg.T, np.linalg.pinv(sigma)), arg)
        return 1/(np.sqrt(((2*np.pi)**n)*np.linalg.det(sigma))) *np.exp(-0.5*term)
    
    def clusters_likelihood(self, x, statistics):
        """
        Calculate the likelihood of a sample with a gaussian mixture distribution.
        Args:
            x: Sample.
            statistics: List of dictionaries containing mean, the covariance matrix and the
                cluster probability for each cluster of the gaussian mixture.
        Returns:
            The gaussian mixture likelihood of the sample.
        """
        p_x_c = []
        for c in statistics:
            p_x_c.append(
                c['pi_k'] * self.likelihood(x, c['mean'], c['sigma'])
            )
        return np.sum(p_x_c)

    def bayes_decision_index(self, x):
        """
        Calculate the bayes index as a ratio between the likelihoods of a sample with the two classes
        of the system.
        Args:
            x: The Sample.
        Returns:
            The bayes index.
        """
        return self.clusters_likelihood(x, self.statistics_c1) / self.clusters_likelihood(x, self.statistics_c2)

    def gaussian_classifier(self, x):
        """
        Classifies a sample according to its bayes index.
        Args:
            x: Sample.
        Returns:
            The predicted class of the sample.
        """
        return self.labels['class_1'] if self.bayes_decision_index(x) >= self.k else self.labels['class_2']
    
    def predict(self, x):
        """
        Batch predict of the gaussian mixture model classifier.
        Args:
            x: Sample matrix.
        Returns:
            Predictions array for each sample.
        """
        predictions = []
        for sample in tqdm(x):
            predictions.append(self.gaussian_classifier(sample))
        return predictions
    
    def predict_indexes(self, x):
        predictions = []
        for sample in tqdm(x):
            predictions.append(self.bayes_decision_index(sample))
        return predictions

Alguns experimentos interessantes a serem observados com os dados utilizados, e o classificador por mistura de gaussianas. Vamos variar a quantidade de clusters utilizados para segmentar as classes e acompanhar a performance do modelo, particularmente o F1-score.

In [ ]:
scores = []
for k in range(1, 21):
    gm = GaussianMixture(clusters=(k, k))
    gm.fit(x, y)
    pred = gm.predict(x_test)
    scores.append(
        (
            k, 
            f1_score(y_test, pred, average='weighted'), 
            recall_score(y_test, pred, average='weighted'), 
            precision_score(y_test, pred, average='weighted')
        )
    )
    

In [ ]:
scores = np.array(scores)
plt.figure()
plt.title('Pontuações de performance do classificador')
plt.ylabel('F1-score')
plt.xlabel('Número de clusters na mistura')
plt.plot(scores[:,0], scores[:,1], 'o-', label='F1-score')
plt.plot(scores[:,0], scores[:,3], 'x-', label='Precision')
plt.plot(scores[:,0], scores[:,2], '.-', label='Recall')
plt.legend()
plt.show()

É interessante notar como, à medida que aumentamos a quantidade de clusters utilizados pelo k-means para descrever os dados, em cada classe, aumentamos o score de classificação  dos dados. Isto é esperado, uma vez que, para uma quantidade maior de clusters, a discriminação dos dados em cada classe torna-se melhor, como esperado do método de mistura de gaussianas. Nota-se, também como, a partir de um determinado número de clusters utilizados, o score não apresenta uma evolução notável, uma vez que segmentar mais os dados em números maiores de agrupamentos não adiciona discriminibilidade às classes, e portanto não adiciona ganho em performance.

Adotando-se, portanto, um número de 10 clusters para os dados sintéticos, temos o seguinte desempenho da rede implementada.

In [ ]:
gm = GaussianMixture(clusters=(10, 10))
gm.fit(x, y)

pred = gm.predict(x_test)

print(classification_report(y_test, pred))
fpr, tpr, _ = roc_curve(y_test, pred)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

e a superfície de separação

In [ ]:
grid_size=200
grid_range=(-15, 15)
x_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
y_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
x1, x2 = np.meshgrid(x_lab, y_lab)
x_grid = np.transpose(np.vstack([x1.flatten(), x2.flatten()]))

y_hat = np.array(gm.predict_indexes(x_grid))
y_hat = y_hat.reshape([grid_size,grid_size])

In [ ]:
plt.plot(x_test[y_test==0,0], x_test[y_test==0,1], '.', label='Classe 1')
plt.plot(x_test[y_test==1,0], x_test[y_test==1,1], '.', label='Classe 2')
plt.contour(x1, x2, y_hat, levels=[1], colors=('black',), linewidths=(2.5,))
plt.show()

Implementando funções de Estimador de Densidade de Kernel (KDE):

In [ ]:
class KDE:
    
    def __init__(self):
        self.labels = {}
        self.X_1 = None
        self.X_2 = None
        self.h = None
        self.k = None
    
    def kernel(self, x):
        return 1 / np.sqrt(2*np.pi) * np.exp(-0.5 * x*x)
    
    def fit(self, x_train, y_train, std=None):
        labels = list(Counter(y_train).keys())
        self.X_1 = x_train[y_train == labels[0]]
        self.X_2 = x_train[y_train == labels[1]]
        
        self.labels = {'class_1': labels[0], 'class_2': labels[1]}
        
        p_class_1 = x_train[y_train == labels[0], :].shape[0]
        p_class_2 = x_train[y_train == labels[1], :].shape[0]

        self.k = p_class_1 / p_class_2
        
        self.h_1 = self.estimate_h(self.X_1, std)
        self.h_2 = self.estimate_h(self.X_2, std)
    
    def density_estimation(self, x, h, data):
        kernel = [np.prod(1/h * self.kernel((x-sample_x)/h)) for sample_x in data]
        return np.sum(kernel) / len(kernel)
    
    def estimate_h(self, data, std):
        if std:
            return np.linalg.norm(1.06 * std * np.power(data.shape[0], -0.2))
        return np.linalg.norm(1.06 * np.std(data, axis=0) * np.power(data.shape[0], -0.2))
        
    
    def bayes_decision_index(self, x):
        """
        Calculate the bayes index as a ratio between the likelihoods of a sample with the two classes
        of the system.
        Args:
            x: The Sample.
        Returns:
            The bayes index.
        """
        return self.density_estimation(x, self.h_1, self.X_1) / self.density_estimation(x, self.h_2, self.X_2)

    def gaussian_classifier(self, x):
        """
        Classifies a sample according to its bayes index.
        Args:
            x: Sample.
        Returns:
            The predicted class of the sample.
        """
        return self.labels['class_1'] if self.bayes_decision_index(x) >= self.k else self.labels['class_2']
    
    def predict(self, x):
        """
        Batch predict of the gaussian mixture model classifier.
        Args:
            x: Sample matrix.
        Returns:
            Predictions array for each sample.
        """
        predictions = []
        for sample in tqdm(x):
            predictions.append(self.gaussian_classifier(sample))
        return predictions
    
    def predict_indexes(self, x):
        predictions = []
        for sample in tqdm(x):
            predictions.append(self.bayes_decision_index(sample))
        return predictions
        

In [ ]:
kde = KDE()
kde.fit(x, y, 0.5)

pred = kde.predict(x_test)

print(classification_report(y_test, pred))
fpr, tpr, _ = roc_curve(y_test, pred)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

In [ ]:
grid_size=100
grid_range=(-15, 15)
x_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
y_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
x1, x2 = np.meshgrid(x_lab, y_lab)
x_grid = np.transpose(np.vstack([x1.flatten(), x2.flatten()]))

y_hat = np.array(kde.predict_indexes(x_grid))
y_hat = y_hat.reshape([grid_size,grid_size])

plt.plot(x_test[y_test==0,0], x_test[y_test==0,1], '.', label='Classe 1')
plt.plot(x_test[y_test==1,0], x_test[y_test==1,1], '.', label='Classe 2')
plt.contour(x1, x2, y_hat, levels=[1], colors=('black',), linewidths=(2.5,))
plt.show()

## Testando métodos para o dataset de expressão gênica

Houve problemas na aplicação das soluções de classificação no dataset disponível de expressão gênica.

### Carregando os dados

In [ ]:
def clean_dataset(data, normalized=False, scaler=None):
    df = data.copy()
    columns = [c for c in df.columns if 'call' in c]
    df = df.set_index('Gene Accession Number')
    df.index.name = None
    df = df.drop(columns + ['Gene Description'], axis=1).transpose()
    df.index = df.index.astype(int)
    
    if not normalized:
        return df, None
    
    x = df.values
    
    if scaler is not None:
        minmax = scaler
    else:
        minmax = MinMaxScaler()
        minmax.fit(x)
        
    x_scaled = minmax.transform(x)
    df.loc[:,:] = x_scaled
    return df, minmax

path = os.path.join('.', 'data_set_ALL_AML_train.csv')
train, scaler = clean_dataset(pd.read_csv(path))

path = os.path.join('.', 'data_set_ALL_AML_independent.csv')
test, _ = clean_dataset(pd.read_csv(path), scaler=scaler)

df = pd.concat([train, test], copy=False)

path = os.path.join('.', 'actual.csv')
cf_label = pd.read_csv(path)
cf_label = cf_label.set_index('patient')
cf_label.index.name = None

df = df.join(cf_label)

x_train, x_test, y_train, y_test = train_test_split(df.drop('cancer', axis=1), df.cancer, test_size=0.4)

mean_1 = x_train.loc[y_train == 'AML', :].mean().to_numpy()
mean_2 = x_train.loc[y_train == 'ALL', :].mean().to_numpy()

std_1 = x_train.loc[y_train == 'AML', :].std().to_numpy()
std_2 = x_train.loc[y_train == 'ALL', :].std().to_numpy()

P = np.abs((mean_1 - mean_2)) / (std_1 + std_2)
features = np.argsort(P)[-50:]

x_train = x_train.iloc[:, features].to_numpy()
x_test = x_test.iloc[:, features].to_numpy()


Relizando uma análise dos dados, um primeiro problema é a quantidade de dados disponíveis para cada amostra. No conjunto de treinamento, temos as seguintes configurações de amostras para cada classe:

In [ ]:
print('Quantidade de pacientes para a classe ALL: ', x_train[y_train == 'AML', :].shape[0])
print('Quantidade de pacientes para a classe AML: ', x_train[y_train == 'ALL', :].shape[0])

Para o caso do classificador com mistura de gaussianas, isto já limita a quantidade de clusters que podemos considerar para a segmentação das classes, uma vez que uma alta quantidade de clusters pode acabar por gerar centróides duplicados ou clusters com matriz de covariância unitária (tendo emvista clusters com apenas uma única amostra). Além disso, ao avaliarmos o determinante da matriz de covariância entre as amostras das classes, utilizada no cálculo da gaussiana multivariável:

In [ ]:
sigma_1 = np.cov(x_train[y_train == 'AML', :].T)
sigma_2 = np.cov(x_train[y_train == 'ALL', :].T)
print(f'Determinante matriz de covariâncias da classe 1 (ALL) = {np.linalg.det(sigma_1)}')
print(f'Determinante matriz de covariâncias da classe 2 (AML) = {np.linalg.det(sigma_2)}')

Um determinante nulo implica, além de uma matriz não invertível, um  problema no cálculo da verossimilhança, onde o determinante da matriz de covariâncias aparece como denominador. Com isso, a verossimilhança de uma amostra para esta classe tende a infinito, e portanto, para qualquer amostra prevista, seu resultado será esta classe. Isto será visto nos testes a seguir.

In [ ]:
gm_ge = GaussianMixture(clusters=(2, 2))
gm_ge.fit(x_train, y_train)

pred = gm_ge.predict(x_test)

print(classification_report(y_test, pred))
y_test_1 = [1 if s == 'AML' else 0 for s in y_test]
pred_1 = [1 if s == 'AML' else 0 for s in pred]
fpr, tpr, _ = roc_curve(y_test_1, pred_1)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

In [ ]:
kde = KDE()
kde.fit(x_train, y_train)

pred = kde.predict(x_test)

print(classification_report(y_test, pred))
y_test_1 = [1 if s == 'AML' else 0 for s in y_test]
pred_1 = [1 if s == 'AML' else 0 for s in pred]
fpr, tpr, _ = roc_curve(y_test_1, pred_1)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

Como visto das predições acima, tanto para mistura de gaussianas quanto para KDE, os desempenhos não foram satisfatórios. Ambos previram apenas amostras para a classe ALL, como comentado anteriormente. Se observarmos a curva ROC, vemos que seu desempenho é similar ao de um classificador aleatório. 

Este desempenho se deu analisando as 50 melhores variáveis considerando o critério de correlação que o paper correspondente aplicou. Vamos fazer um pouco diferente agora, e vamos fazer uso de PCA para selecionar duas melhores componentes considerando todo o conjunto de dados. O modelo do PCA será treinado utilizando o conjunto de treinamento, e então aplicado ao conjunto de teste, para evitar a inserção de viés nos dados testados.

In [ ]:
path = os.path.join('.', 'data_set_ALL_AML_train.csv')
train, scaler = clean_dataset(pd.read_csv(path), normalized=False)

path = os.path.join('.', 'data_set_ALL_AML_independent.csv')
test, _ = clean_dataset(pd.read_csv(path), scaler=scaler, normalized=False)

df = pd.concat([train, test], copy=False)

path = os.path.join('.', 'actual.csv')
cf_label = pd.read_csv(path)
cf_label = cf_label.set_index('patient')
cf_label.index.name = None

df = df.join(cf_label)

pca = PCA(n_components=2)

x_train, x_test, y_train, y_test = train_test_split(df.drop('cancer', axis=1), df.cancer, test_size=0.4)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)


In [ ]:
sigma_1 = np.cov(x_train[y_train == 'AML', :].T)
sigma_2 = np.cov(x_train[y_train == 'ALL', :].T)
print(f'Determinante matriz de covariâncias da classe 1 (ALL) = {np.linalg.det(sigma_1)}')
print(f'Determinante matriz de covariâncias da classe 2 (AML) = {np.linalg.det(sigma_2)}')

De maneira similar ao que fizemos anteriormente para o conjunto de dados sintéticos, vamos avaliar o desempenho do classificador para diferentes números de clusters utilizados na mistura de gaussianas, por classe.

In [ ]:
scores = []
for k in range(1, 11):
    gm = GaussianMixture(clusters=(k, k))
    gm.fit(x_train, y_train)
    pred = gm.predict(x_test)
    scores.append(
        (
            k, 
            f1_score(y_test, pred, average='weighted'), 
            recall_score(y_test, pred, average='weighted'), 
            precision_score(y_test, pred, average='weighted')
        )
    )
    
scores = np.array(scores)
plt.figure()
plt.title('Pontuações de performance do classificador')
plt.ylabel('F1-score')
plt.xlabel('Número de clusters na mistura')
plt.plot(scores[:,0], scores[:,1], 'o-', label='F1-score')
plt.plot(scores[:,0], scores[:,3], 'x-', label='Precision')
plt.plot(scores[:,0], scores[:,2], '.-', label='Recall')
plt.legend()
plt.show()

Utilizando portanto 2 clusters por classe para dividir os dados, temos.

In [ ]:
gm_ge = GaussianMixture(clusters=(2, 2))
gm_ge.fit(x_train, y_train)

pred = gm_ge.predict(x_test)

print(classification_report(y_test, pred))
y_test_1 = np.array([1 if s == 'AML' else 0 for s in y_test])
pred_1 = np.array([1 if s == 'AML' else 0 for s in pred])
fpr, tpr, _ = roc_curve(y_test_1, pred_1)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

A superfície de separação, e os dados de teste classficados de acordo podem ser vistos a seguir.

In [ ]:
grid_size=200
grid_range=(-100000, 100000)
x_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
y_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
x1, x2 = np.meshgrid(x_lab, y_lab)
x_grid = np.transpose(np.vstack([x1.flatten(), x2.flatten()]))

y_hat = np.array(gm_ge.predict_indexes(x_grid))
y_hat = y_hat.reshape([grid_size,grid_size])

In [ ]:
plt.figure()
plt.plot(x_test[y_test_1==0,0], x_test[y_test_1==0,1], '.', label='Classe 1')
plt.plot(x_test[y_test_1==1,0], x_test[y_test_1==1,1], '.', label='Classe 2')
plt.contour(x1, x2, y_hat, levels=[1], colors=('black',), linewidths=(2.5,))
plt.title('Superfície de separação e dados de teste')
plt.show()

Os dados de treinamento utilizados para a obtenção da superfície podem ser vistos juntamente com a mesma nos gráficos a seguir.

In [ ]:
plt.figure()
y_train_1 = np.array([1 if s == 'AML' else 0 for s in y_train])
plt.plot(x_train[y_train_1==0,0], x_train[y_train_1==0,1], '.', label='Classe 1')
plt.plot(x_train[y_train_1==1,0], x_train[y_train_1==1,1], '.', label='Classe 2')
plt.contour(x1, x2, y_hat, levels=[1], colors=('black',), linewidths=(2.5,))
plt.title('Superfície de separação e dados de treinamento')
plt.show()

No caso do modelo KDE, o desempenho de treinamento é:

In [ ]:
kde_ge = KDE()
kde_ge.fit(x_train, y_train)

pred = kde_ge.predict(x_train)

print(classification_report(y_train, pred))
y_test_1 = np.array([1 if s == 'AML' else 0 for s in y_train])
pred_1 = np.array([1 if s == 'AML' else 0 for s in pred])
fpr, tpr, _ = roc_curve(y_test_1, pred_1)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

E o desempenho obtido nos dados de teste:

In [ ]:
kde_ge = KDE()
kde_ge.fit(x_train, y_train)

pred = kde_ge.predict(x_test)

print(classification_report(y_test, pred))
y_test_1 = np.array([1 if s == 'AML' else 0 for s in y_test])
pred_1 = np.array([1 if s == 'AML' else 0 for s in pred])
fpr, tpr, _ = roc_curve(y_test_1, pred_1)
plt.figure() 
plt.title('ROC')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.plot(fpr, tpr)
plt.show()

A superfície de separação obtida pelo modelo pode ser vista plotada a seguir.

In [ ]:
grid_size=100
grid_range=(-100000, 100000)
x_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
y_lab = np.linspace(grid_range[0], grid_range[1], num=grid_size)
x1, x2 = np.meshgrid(x_lab, y_lab)
x_grid = np.transpose(np.vstack([x1.flatten(), x2.flatten()]))

y_hat = np.array(kde_ge.predict_indexes(x_grid))
y_hat = y_hat.reshape([grid_size,grid_size])
plt.figure()

plt.plot(x_test[y_test_1==0,0], x_test[y_test_1==0,1], '.', label='Classe 1')
plt.plot(x_test[y_test_1==1,0], x_test[y_test_1==1,1], '.', label='Classe 2')
plt.contour(x1, x2, y_hat, levels=[1], colors=('black',), linewidths=(2.5,))
plt.show()